In [1]:
# GLOBAL VARIABLES. BE SURE NOT TO OVERWRITE THEM
D = 5 # Amount of documents
V = 10 # Size of the vocabulary
M = 10 # Maximum amount of same word repetition in a document
k = 5 # Amount of topics
gamma = 0.05

## IMPORTANT: Please use static random seeds in **EVERY** cell where you use a random function, so that the result does **NOT** change at every run.

# 1. ARTIFICIAL DATA

### Task:

You must implement an algorithm that generates an artificial *corpus*, and return also a graph G and a correlation matrix Sigma.

In [2]:
import numpy as np
from scipy.stats import bernoulli
#!pip install sklearn
from sklearn.datasets import make_sparse_spd_matrix

In [3]:
# Transformation functions (deterministic)

def update_Theta(Theta, H):
    for d in range(D):
        Theta[d] = np.exp(H[d]) / np.sum(np.exp(H[d]), axis=0)
    print('Success: Theta transformed from H')
    return Theta

def update_E(E, Z):
    for topic in range(1, len(E)):
        E[topic, :] = np.sum(Z == topic, axis=2).sum(axis=1)
    print('Success: E transformed from Z')
    return E

def update_C(C, Z):
    for topic in range(1, len(C)):
        C[topic, :] = np.sum(Z == topic, axis=2).sum(axis=0)
    print('Success: C transformed from Z')
    return C

def update_B(B, C):
    # Note this is the transformation from C
    for topic in range(0, len(B)):
        B[topic] = C[topic] / sum(C[topic])
    print('Success: B transformed from C')
    return B

def update_Sigma(K):
    Sigma = np.linalg.inv(K)
    print('Success: Sigma transformed from K')
    return Sigma

In [4]:
# Random / Generating functions

def build_topic_distribution(seed):
    np.random.seed(seed)
    distribution = np.random.random(V)
    return distribution / distribution.sum()

def sample_B(seed):
    # B is the matrix whose rows are the distribution of topic i over the vocabulary
    # Each row means : for each topic i we have the probability of word i to occur
    # Using what Kanthavel did before
    b = np.empty((k,V))
    np.random.seed(seed)
    for i in range(k):
        b[i,:] = build_topic_distribution(seed)
    return b

def sample_G(k, gamma, seed):  # Won't update Sigma automatically anymore
    # Bernoulli for G
    # generate a random adjacency matrix
    np.random.seed(seed)
    matrix = np.array([[int(bernoulli.rvs(p=gamma, size=1)) for i in range(k)] for j in range(k)])
    for i in range(k):
        matrix[i][i] = 0
    for i in range(k):
        for j in range(k):
            matrix[j][i] = matrix[i][j]
    return matrix

def sample_K(k, seed):  # Won't update Sigma automatically anymore
    # I can build K for using make_sparse_spd_matrix from sklearn.datasets for example
    np.random.seed(seed)
    K = make_sparse_spd_matrix(k, alpha=0.95, norm_diag=False, smallest_coef=0.1, largest_coef=0.9, random_state=None)
    return K

def sample_H(Sigma, k, seed):  # Won't update Theta automatically anymore
    # Multivariate Normal
    mu = np.zeros(k)
    np.random.seed(seed)
    H = np.random.multivariate_normal(mu, Sigma, k)
    return H

In [5]:
# Main Simulator Class
class Simulator:
    
    # Remember we will have indexes starting from 0 so all max are -=1
    
    def __init__(self, D, V, k, seed):
        # Create zero matrices for all possible matrices
        self.W = np.zeros((D, V))  # matrix of D×V where Wdn is counter of appearances of the word n in document d
        self.B = np.zeros((k, V))  # matrix of kxV where Bz is the parameter vector of the distribution for the z-th topic
        self.C = np.zeros((k, V))  # matrix of kxV where Cz is the count vec of sampled topics over each word for all docs
        self.E = np.zeros((D, k))  # matrix of Dxk where Ed is the count vec of sampled drawings for topic z over all words for each doc
        self.H = np.zeros((D, k))  # H_d is eta_d
        self.Theta = np.zeros((D, k))  # This is just a transformation of H
        self.G = np.zeros((k, k))  # Adjacency Matrix (Check also python package "networkx" for graph objects!)
        self.K = np.zeros((k, k))  # Precision matrix of G
        self.Sigma = np.zeros((k, k))  # Inverse of K
        self.Z = None  # Z shouldn't be created before W is sampled (depends if generated or read)
        self.seed = seed  # Random seed
        
    def get_M(self):
        # Ref: https://numpy.org/doc/stable/reference/generated/numpy.matrix.max.html
        return int(self.W.max())
    
    # Generations
    def generate_W_from_M(self, M):
        # We use M as the maximum repetitions of words
        # Hence, for each word in V in each document, we throw a randint(0,M) for the generative process
        if M == 0:
            raise Exception('Error: Input 0 for the chosen method')

        np.random.seed(self.seed)
        for d in range(D):
            for n in range(V):
                self.W[d,n] = np.random.randint(0,M)
        print('Success: W generated from M')
        
    def generate_W_from_Z(self):  # TODO: This is deterministic, maybe we can factor it out with the other functions?
        
        if self.Z is None or np.sum(self.Z, axis=2).sum(axis=1).sum(axis=0) == 0:
            raise Exception('Error: Z matrix is None or 0')
        
        # We just need the lengths of Z[d,n]
        # Since that gives the repetitions of word n in doc d
        for d in range(D):
            for n in range(V):
                # This gives the index of the element next to the end of Z
                # Hence its already decreased to give the amounts
                derp = np.where(self.Z[d,n] == -1)
                rep = derp[-1][0]
                self.W[d, n] = int(rep)
        print('Success: W generated from Z')
        
    def generate_Z_from_W(self):
        m = self.get_M()
        self.Z = - np.ones((D, V, int(m)))
        # Multinomial drawing from Theta, because it has to be normalized
        # Ref https://numpy.org/doc/stable/reference/random/generated/numpy.random.multinomial.html
        # np.random.multinomial(1, self.Theta[d], size=1)
        # This will give a canonical vector over k
        
        np.random.seed(self.seed)
        if np.sum(self.Theta, axis=1).sum(axis=0) == 0:
            raise Exception('Error: Theta matrix 0')
        
        for d in range(D):
            for n in range(V):
                for m in range(self.W[d,n]):
                    mult = np.random.multinomial(1, self.Theta[d], size=1)
                    self.Z[d,n,m] = int(np.where(mult == 1))
        print('Success: Z generated from Theta')
    
    def generate_Z_from_M(self, M):
        self.Z = - np.ones((D, V, M))
        # Multinomial drawing from Theta, because it has to be normalized
        # Ref https://numpy.org/doc/stable/reference/random/generated/numpy.random.multinomial.html
        # np.random.multinomial(1, self.Theta[d], size=1)
        # This will give a canonical vector over k
        np.random.seed(self.seed)
        if np.sum(self.Theta, axis=1).sum(axis=0) == 0:
            raise Exception('Error: Theta matrix 0')
        for d in range(D):
            for n in range(V):
                for m in range(np.random.randint(0, M)):
                    mult = np.random.multinomial(1, self.Theta[d], size=1)
                    self.Z[d,n,m] = np.where(mult == 1)[-1]
        print('Success: Z generated from Theta')
    
    # Transformations
    def update_Theta(self):
        self.Theta = update_Theta(self.Theta, self.H)
    
    def update_E(self):
        self.E = update_E(self.E, self.Z)
    
    def update_C(self):
        self.C = update_C(self.C, self.Z)
        
    def update_B(self):
        self.B = update_B(self.B, self.C)
    
    def update_Sigma(self):
        self.Sigma = update_Sigma(self.K)
    
    # Initializing with real data
    # def save_W()
    
    # Priors
    def sample_B(self):
        self.B = sample_B(self.seed)
        
    def sample_GK(self, gamma):  # Here we can update Sigma automatically
        self.G = sample_G(k, gamma, self.seed)
        self.K = sample_K(k, self.seed)
        self.update_Sigma()
    
    def sample_H(self):  # Here we can update Theta automatically
        self.H = sample_H(self.Sigma, k, self.seed)
        self.update_Theta()
    
    def generate_all_data(self):
        # TODO: This should run all relevant methods one after the other in order to fully populate all data matrixes
        # self.generate something
        # self.update something
        # ecc
        pass

# 2 MC SAMPLER

# 2.1 MCMC Sampling

### Task:

You must implement a function that receives matrices $W$, $\Theta_{i+1}$ and $B_i$ and generates the next $Z_{i+1}$ and $B_{i+1}$.

In [6]:
def binary_search(sequence, item):
    begin_index = 0
    end_index = len(sequence)-1
    
    if sequence[begin_index] <= item <= sequence[end_index]:
        while begin_index < end_index - 1:  # Finish when the list has 2 items: Begin and end
            midpoint = (end_index + begin_index)//2
            midpoint_value = sequence[midpoint]
            if midpoint_value < item:
                begin_index = midpoint
            else:
                end_index = midpoint
        if sequence[begin_index] == item:
            return begin_index
        elif item <= sequence[end_index]:
            return end_index
    else:
        return -1

In [7]:
def MC_sample_Z(Z, W, Theta, B, E, C):  # D, k are global variables
    for d in range(D):
        # computing N_d
        N_d = int(np.sum(W[d]))
        for i in range(N_d):
            v = V[i]
            I_di = W[d][i]
            for j in range(I_di):
                z_hat = Z[d][i][j]
                
                E[d][z_hat] = max(0, E[d][z_hat]-1)
                
                C[z_hat][v] = max(0, C[z_hat][v]-1)
                
                Rho = [0]  # Needs to start from zero to have the interval to fall into topic 1
                Rho_z = 0
                
                for z in range(k):
                    # Compute the denominator sum
                    C_vk = 0
                    for b in range(N_d):
                        if b != i:
                            C_vk += C[z][b]
                    # Compute the upper limits of the topic probabilities
                    Rho_z += (E[d][z] + Theta[d])*(C[z][v] + B[z])/(sum(C_vk)+V*B[z])
                    Rho.append(Rho_z)
                    
                u = np.random.uniform(0, Rho[-1])
                z_hat = binary_search(Rho, u)
                E[d][z_hat] += 1
                C[z_hat][v] += 1
                Z[d][i][j] = z_hat
    # Note that we directly modify Z since the update per topic helps for the next iteration 
    return Z

# 2.2 Metropolis-Hastings MC Sampling

### Task:

You must implement a function that receives matrices $E_i$, $K_i$ and vector $\mu$ and generates the next $H_{i+1}$.


. $E$ matrix of $D \times k$ where $E_d$ is the $k$-dim vector of counts of sampled drawings for the $z$-th topic over all words for each document

. $K$ matrix of $k \times k$ representing the precision matrix associated to the graph $G$

. $\mu = 0$

. $H$ matrix of $D \times k$ where $H_d = \eta_d$ is the $k$-dim vector of the topic prevalences over document $d$

In [8]:
import numpy as np
import numpy.linalg

In [9]:
def sampled_distribution_kernel(eta, K, E):
    k = eta.shape[0]
    eta_K_eta = -0.5 * eta.dot(K.dot(eta))
    E_eta = E.dot(eta)
    sum_eta_pow_k = np.sum(np.exp(eta)) ** k
    return np.exp(eta_K_eta + E_eta) / sum_eta_pow_k

In [10]:
def MC_sample_H(E, Sigma, H_current=None, burn_in=100, seed=None):
    
    np.random.seed(seed)
    
    K = np.linalg.inv(Sigma)
    
    D, k = E.shape  # Number of documents, Number of topics
    
    if H_current is None:
        H_current = np.zeros((D, k))
    
    H_sampled = np.zeros((D, k))
    
    for d in range(D):  # Iterating over each document
        eta_current = H_current[d]
        E_d = E[d]
        for iteration in range(burn_in + 1):
            
            # Sampling proposed eta from multivariate normal (q "proposal density")
            eta_prop = np.random.multivariate_normal(eta_current, Sigma)
            
            # Compute acceptance probability
            alpha = min(1, sampled_distribution_kernel(eta_prop, K, E_d) / sampled_distribution_kernel(eta_current, K, E_d))
            
            if alpha == 1 or np.random.uniform(0.0, 1.0) < alpha:
                eta_current = eta_prop
            
        H_sampled[d] = eta_current
    
    return H_sampled

# 2.3 BDMCMC Sampling

### Task:

You must implement a function that receives matrices $W$, $Z_{i+1}$ and $H_{i+1}$ and generates the next $G_{i+1}$ and $K_{i+1}$.

In [11]:
"""
def BDMCMC_Sampling(W, Z, H):
    #update_G(W,Z,H,K,G)
    #update_K()
    return (G,K)
"""

'\ndef BDMCMC_Sampling(W, Z, H):\n    #update_G(W,Z,H,K,G)\n    #update_K()\n    return (G,K)\n'

In [12]:
def MC_sample_K(G, b, shape_matrix):
    # save G to csv
    # save shape_matrix to csv
    # (OR pass them as parameters to RScript)
    # call R script using python.subprocess
    # read the results from csv
    # (OR get the result back from R script)
    # return resulting K
    pass

In [13]:
def MC_sample_G(W, Z, H, K, G, E):

    N = G.shape[0]
    delta_K = 0
    beta_K = 0

    death_rates = np.zeros((k,k))
    birth_rates = np.zeros((k,k))

    PrHK = lambda K,H:      K.size**(n/2) * np.exp(-0.5*np.trace(np.matmul(np.matmul(K, np.transpose(H)), H)))
    PrK_G = lambda K,G,D,b: K.size**(b+D-2) * np.exp(-0.5*np.trace(np.matmul(S + np.matmul(np.transpose(H), H), K)))
    PrG = lambda gamma, E:  (gamma/(1-gamma))**(E.size)

    PrKG_H = lambda K,G,H,D,b,gamma,E: PrHK(K,H)*PrK_G(K,G,D,b)*PrG(gamma,E)

    Pr_init = PrKG_H(K,G,H,D,k-1,gamma,E)

    for i in range(N):
        for j in range(i+1, N):
            if G[i,j]:
                G_loop = G.copy()
                G_loop[i,j], G_loop[j,i] = 0
                #technically, we should compute K_loop here...
                Pr_loop = PrKG_H(K,G_loop,H,D,b,gamma,E)

                death_rate = Pr_loop/Pr_init

                if death_rate > 1:
                    death_rate = 1
                death_rates[i,j], death_rates[j,i] = death_rate, death_rate
                delta_K += death_rate

            else:
                G_loop = G.copy()
                G_loop[i,j], G_loop[j,i] = 1
                #technically, we should compute K_loop here...
                Pr_loop = PrKG_H(K,G_loop,H,D,b,gamma,E) 

                birth_rate = Pr_loop/Pr_init

                if birth_rate > 1:
                    birth_rate = 1
                birth_rates[i,j], birth_rates[j,i] = birth_rate, birth_rate
                beta_K += birth_rate
    
    W = 1/(beta_K + delta_K)

    pr_death = W*death_rates
    pr_birth = W*birth_rates

    for i in range(N):
        for j in range(i+1, N):
            if pr_death[i,j] > 0.5:
                G[i,j], G[j,i] = 0,0
            elif pr_birth[i,j] > 0.5:
                G[i,j], G[j,i] = 1,1

    return G


# TESTING

### Data Generation

In [14]:
test0 = Simulator(D, V, k, seed=1996)
test0.sample_GK(gamma)  # Will get G, K, Sigma
test0.sample_H()  # Will get H, Theta from Sigma
test0.generate_Z_from_M(M)  # Will get Z from Theta
test0.sample_B()  # Will get B
test0.generate_W_from_M(M)  # Will get W
test0.update_E()
test0.generate_W_from_Z()

# Note for debugging:
# After updating the class, you need to re run the class object initialization
# Else for some reason it uses the outdated code

# K:
# (well yeah ;) jupyter is an interactive programming interface
# it only runs the code when you tell it to do so.
# If you define class C, then create object c = C(), then redefine class C, 
# object c will still be using the old definition of C)

Success: Sigma transformed from K
Success: Theta transformed from H
Success: Z generated from Theta
Success: W generated from M
Success: E transformed from Z
Success: W generated from Z


In [15]:
test0.W

array([[1., 5., 8., 3., 5., 8., 8., 7., 4., 9.],
       [3., 8., 7., 9., 4., 2., 0., 6., 5., 6.],
       [9., 6., 0., 8., 8., 9., 5., 7., 1., 5.],
       [9., 9., 0., 0., 7., 1., 6., 3., 6., 0.],
       [5., 7., 0., 7., 5., 2., 4., 9., 6., 4.]])

In [16]:
# It would be nice to have it all in one function
test1 = Simulator(D, V, k, seed=1979)
test1.generate_all_data()
test1.W

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

### MC Samplers

In [17]:
MC_sample_Z(test0.Z, test0.W, test0.Theta, test0.B, test0.E, test0.C)

TypeError: 'int' object is not subscriptable

In [18]:
MC_sample_H(test0.E, test0.Sigma, seed=1984)

/tmp/ipykernel_7143/1325047619.py:6: RuntimeWarning: overflow encountered in exp
  return np.exp(eta_K_eta + E_eta) / sum_eta_pow_k
/tmp/ipykernel_7143/2872664781.py:23: RuntimeWarning: invalid value encountered in double_scalars
  alpha = min(1, sampled_distribution_kernel(eta_prop, K, E_d) / sampled_distribution_kernel(eta_current, K, E_d))


array([[-0.4101077 , -1.06819239,  0.22497691,  0.95175126,  0.27635825],
       [10.97143063, 10.84444124,  7.078654  , 10.29164726,  7.29824537],
       [ 8.14881642,  9.58360464, 14.84217707, 12.12033855, 16.66967783],
       [ 5.75354115,  4.25074912,  4.07104324,  1.2310418 ,  3.99881796],
       [16.75082073, 10.87129595,  4.49843587, 15.84988715,  9.00874339]])

In [20]:
MC_sample_G(test0.W, test0.Z, test0.H, test0.K, test0.G, test0.E)

NameError: name 'n' is not defined

In [21]:
b = k - 1  #  Or something... ? 
shape_matrix = np.eye(k)  # Or something... ?
MC_sample_K(test0.G, b, shape_matrix)

# MAIN ALGORITHM

### Generating Data

In [22]:
simulated_data = Simulator(D, V, k, seed=1984)
simulated_data.sample_GK(gamma)  # Will get G, K, Sigma
simulated_data.sample_H()  # Will get H, Theta from Sigma
simulated_data.generate_Z_from_M(M)  # Will get Z from Theta
simulated_data.sample_B()  # Will get B
simulated_data.generate_W_from_M(M)  # Will get W
simulated_data.update_E()
simulated_data.generate_W_from_Z()

Success: Sigma transformed from K
Success: Theta transformed from H
Success: Z generated from Theta
Success: W generated from M
Success: E transformed from Z
Success: W generated from Z


In [23]:
# Input Data:
simulated_data.W

array([[3., 5., 4., 9., 3., 2., 8., 4., 4., 9.],
       [0., 8., 0., 1., 2., 3., 2., 5., 5., 7.],
       [3., 3., 0., 6., 4., 8., 5., 6., 4., 4.],
       [8., 1., 5., 3., 4., 3., 0., 2., 9., 2.],
       [3., 4., 1., 4., 8., 8., 1., 4., 1., 8.]])

# SAMPLER

In [25]:
# # Initial guesses
initial = Simulator(D, V, k, 125)
initial.sample_GK(gamma)
initial.sample_B()
initial.sample_H()
initial.W = simulated_data.W.astype(int)  # Passing data to the simulator
initial.generate_Z_from_W()  # Generating random topic distributions based on real data
initial.update_E()  # Deterministic transformation of Z (counters)
initial.update_C()  # Deterministic transformation of Z (counters)

Success: Sigma transformed from K
Success: Theta transformed from H


TypeError: int() argument must be a string, a bytes-like object or a number, not 'tuple'

In [26]:
max_iteration = 100

# Data
W = simulated_data.W

# Initialization
Sigma = initial.Sigma
K = np.linalg.inv(Sigma)
B = initial.B  # MATTEO, PLEASE HELP US!!!
Theta = initial.Theta
Z = initial.Z
E = initial.E
C = initial.C
G = initial.G

for iteration in range(max_iteration):
    
    # Step 1
    Z = MC_sample_Z(Z, W, Theta, B, E, C)
    E = update_E(E, Z)  # get E from Z
    C = update_C(C, Z)  # get C from Z ...
    
    # Step 2
    H = MC_sample_H(E, Sigma)
    Theta = update_Theta(Theta, H)  # get Theta from H
    
    
    # Step 3
    G = MC_sample_G(W,Z,H,K,G)
    # Need to define b and shape_matrix!
    K = MC_sample_K(G, b, shape_matrix)
    Sigma = np.inv(K)
    
    # Hope for convergence!
    wrong_edges = np.sum(np.equal(G, simulated_data.G))
    error = np.norm(Sigma - simulated_data.Sigma)
    print(f"At iteration {iteration}, the wrong edges are {wrong_edges} and the error on Sigma is {error}")

TypeError: 'int' object is not subscriptable